# Part 2: Why all these packages?

In the first part of this assignment you learned about `shapely` and `fiona` to represent geospatial vector data and perform geometric operations. From the GeoScripting course you already know the `ogr` package which provides the same functionalities. 

## Python packages `ogr` and `gdal` enable vector and raster data processing

* `ogr` and `gdal` are automatically generated Python bindings (using SWIG) to the C libraries GDAL and OGR 

* So when you use `ogr` and `gdal` classes in Python you are actually executing C code. 

&rarr; Great! C is fast and we can use it from Python.

## So why would we need other Python packages for geo data processing? 

In discussions about Python you will often read statements like this:

![gdal_vs_rasterio](img/gdal_vs_rasterio.png)

[Source: GitHub Issue - rasterio vs python gdal #11](https://github.com/inbo/niche_vlaanderen/issues/11)

### In this assignment we will focus on the following questions: 

1. What does _[Pythonic](https://docs.python-guide.org/writing/style/)_ mean? Why is it suitable for scientists?
2. Why are `ogr` and `gdal` not _Pythonic_ enough? 
3. What are the advantages and disadvantages of using ogr, shapely or your own implementation? 

## Comparing ogr, shapely and pure Python

In this first section of the assignment we will **compare three different ways of representing a polygon in Python.**

1. A very simplified, **pure Python implementation** written by yourself.
2. The **`Geometry` class provided by `ogr` package**.
3. The **`Polygon` class provided by the `shapely` package.**

#### The three methods should be compared in regard to:
* Execution time
* Programming time (how long did it take to write the code)
* Readability
* Flexibility (how easily can you adapt it to your needs)

### 1. Create your own `Polygon` class in Python

__E 1.1:__ Write a class called `MyPolygon` which takes a list of tuples containing the coordinates of the polygon for initialization. The coordinates should be stored as an attribute of the instance called "coordinates". Use the `test_coordinates` below to test your class. 

In [41]:
test_coordinates = [(1,1),(1,2),(2,2),(2,1),(1,1)]

In [4]:
class MyPolygon:
        
    def __init__(self, coordinates):
        self.coordinates = coordinates
        

In [5]:
my_poly = MyPolygon(test_coordinates)

Test whether the instance of class `Polygon` returns the right value. If there is not error message, everything's fine. 

In [6]:
assert my_poly.coordinates == test_coordinates

__E 1.2:__ Add a method `envelope` which calculates the bounding box of the polygon. The method should return the bounding box as a list containing the coordinates of the bounding box `[min_x, min_y, max_x, max_y]`.

__Note:__ A bounding box is often reffered to as the envelope of a geometry.

In [34]:
class MyPolygon():
        
    def __init__(self, coordinates):
        self.coordinates = coordinates
        
    def envelope(self):
        xs = [i[0] for i in self.coordinates]
        ys = [i[1] for i in self.coordinates]
        return [min(xs), min(ys), max(xs), max(ys)]

In [35]:
my_poly = MyPolygon(test_coordinates)

In [36]:
my_poly_env = my_poly.envelope()
print("minX: {0}, minY: {1}, maxX: {2}, maxY: {3}".format(*my_poly_env))

minX: 1, minY: 1, maxX: 2, maxY: 2


### 2. Calculate bounding box using  `ogr.Geometry`
So you've created a very simple Polygon class in Python with just a few lines of code. Now let's compare your implementation to the one provided by the `ogr` package. 

__E 2.1:__ Use the `ogr` package to create a polygon object using the same `test_coordinates` as above.

In [42]:
import ogr

In [59]:
ogr_poly = ogr.Geometry(ogr.wkbPolygon)
ring = ogr.Geometry(ogr.wkbLinearRing)
for point in test_coordinates:
    ring.AddPoint(point[0], point[1])
ogr_poly.AddGeometry(ring)

0

__E 2.2:__ Calculate the envelope of the polygon. 

In [60]:
ogr_poly.GetEnvelope()

(1.0, 2.0, 1.0, 2.0)

Are the results from your class and the ogr class the same? If not check your code.

__Note:__ Remember that the order of the coordinates of the bounding box returned by `ogr` method is  `[min_x, max_x, min_y, max_y]`. 

### 3. Calculate the bounding box using `shapely` 

Those were a lot of lines just for creating a simple polygon. For this reason, `shapely` was developed. Creating a geometry using shapely works the same way as for our Python implementation. 

__E 3.1:__ Use `shapely` package to create a polygon object using the same `test_coordinates` as above.

In [53]:
from shapely.geometry import Polygon

In [62]:
test_coor = test_coordinates
test_coor.append(test_coordinates[0])
shapely_poly = Polygon(test_coor)

__E 3.2:__ Calculate the bounding box of the shapely polygon using the method `bounds()`. Again, check if your results are the same.

In [63]:
shapely_poly.bounds

(1.0, 1.0, 2.0, 2.0)

### 4. Comparison of execution time
Compare the exectution times of both methods using the magic command `%%timeit`. This function will execute the cell multiple times to get a good estimate of the execution time. 

#### Exection time of bounding box calculation
__E 4.1:__ Compare the exection times of the calculation of the bounding box of all three methods. 

In [89]:
%%timeit
my_poly.envelope()

1.65 µs ± 21.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [90]:
%%timeit
ogr_poly.GetEnvelope()

597 ns ± 11.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [91]:
%%timeit
shapely_poly.bounds

31.4 µs ± 411 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


__Question:__ What do you observe when you compre the execution times of all three methods? Can you explain the difference in execution times?

__Answer:__ 

#### Comparison bounding box creation and object creation

When choosing the most efficient way to calculate something, we also need to consider the _overhead_ of the calculation. The _overhead_ contains all the processing steps that need to be taken as a preparation before the execution of the desired calculation. Depending on the implementation, this can change your decision.  

__E 4.2:__ Measure the execution times of all three implementations __including the object creation.__ 

In [98]:
%%timeit
my_poly = MyPolygon(test_coordinates)
my_poly_env = my_poly.envelope()

6.06 ms ± 96.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [99]:
%%timeit
ogr_poly = ogr.Geometry(ogr.wkbPolygon)
ring = ogr.Geometry(ogr.wkbLinearRing)
for point in test_coordinates:
    ring.AddPoint(point[0], point[1])
ogr_poly.AddGeometry(ring)
ogr_poly.GetEnvelope()

48.2 ms ± 531 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [100]:
%%timeit
shapely_poly = Polygon(test_coor)
shapely_poly.bounds

3.79 ms ± 111 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


__Question:__ What do you observe when you compre the execution times and the object creation of all three polygon implementations? Can you explain the difference in execution times?

__Your Answer:__

### 5. Questions for Discussion

__Q1:__ Based on your results of the exercises, evaluate the three methods in the table below. (Double click the cell, to edit it)


|            | Execution time <br> (fast - slow)| Programming time <br> (fast - slow)| Readability <br> (high - low) | Flexibility  <br> (high - low)       |
|------------|----------------|------------------|-------------|-------------------|
| __DIY Python__ |    fast      |       slow       |   medium      |      low      |
| __OGR__        |   medium   |      slow        |     low     | high |
| __Shapely__    |    slow  |        fast      |     high    | high |

__Q2:__ Answer the questions from the beginning of the notebook. 

1. First, try to answer them on your own. 
2. Then, discuss them in groups or pairs with your peers. 
3. We will discuss the results in the course at the end of today's session. 

Please provide your answers below.

1. What does _[Pythonic](https://docs.python-guide.org/writing/style/)_ mean? Why is it suitable for scientists?
2. Why are `ogr` and `gdal` not _Pythonic_ enough? 
3. What are the advantages and disadvantages of using ogr, shapely or your own implementation for vector data processing? 

__Your answers:__
1. Taking advantage of Python's features to make the code readable, reusable and reproducible
2. ogr and gdal libraries are written in C and are automatically converted into Python thus it doesn't have typical Pythonic structure
3. own implementation – pros: the scientist can write his own function tailored for him; cons: slow programming time, high programming skills are required
    gdal/ogr – pros: high variability of supported data formats, implemented in different softwares; cons: not that pythonic
    shapely – 

### Summary:

* There are different ways to perform geometric manipulations in Python. 
* However, when we do complex spatial analysis, we will need to apply these operations to whole layers and not just single features. 
* To make such analyses possible the `pandas` and `geopandas` packages are used. 

In the last part of the assignment, we will look at the `pandas` an `geopandas` packages to perform more complex spatial analyses. 

&rarr; Continue with [Part 3: Vector Data Analysis using GeoPandas](./02-geopandas.ipynb)

## References

If you would like to take a deeper look at object oriented programming in Python take a look at the following resources: 

* [Object Oriented Programming in Python](https://github.com/TheDigitalCatOnline/thedigitalcatonline.github.com/tree/master/notebooks)
* [Python Tutorial](https://docs.python.org/3/tutorial/classes.html)
* [Abstract Classes](https://docs.python.org/3/library/abc.html#module-abc)
* [Computational Geometry in Python: From Theory to Application](https://www.toptal.com/python/computational-geometry-in-python-from-theory-to-implementation
)

    
